In [1]:
import os
import zipfile
import pandas as pd
import numpy as np
import pathlib
import shapefile
import pyreadstat
from tqdm import tqdm
import pickle

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Setting data folder
working_directory = os.getcwd()
data_folder = '../extracted_files'

In [3]:
# Unzipping and reading the birth recode files
for item in os.listdir(working_directory):
    if item.endswith('.zip') and "BR" in item and "DT" in item:
        print(f"Unzipping: {item}")
        with zipfile.ZipFile(item, 'r') as zip_ref:
            zip_ref.extractall(data_folder)            
BR_data = {}
BR_metadata = {}
for file in pathlib.Path(data_folder).glob('*BR*.DTA'):
    country_code = file.name[:2]
    df, meta = pyreadstat.read_dta(file)
    df = df.assign(country=country_code)
    BR_data[country_code] = df
    BR_metadata[country_code] = meta   
if not BR_data:
    print("No BR_dataframes created. Check if .DTA files are present in the expected pattern.")
else:
    print(f"Created BR_dataframes for {len(BR_data)} countries.")


Created BR_dataframes for 31 countries.


In [4]:
# unzipping and reading the geolocalisation (GE data) for all countries from the data folder
for item in os.listdir(working_directory):
    if item.endswith('.zip') and "GE" in item and "FL" in item:
        print(f"Unzipping: {item}")
        with zipfile.ZipFile(item, 'r') as zip_ref:
            zip_ref.extractall(data_folder)           
GE_data = {}
for file in pathlib.Path(data_folder).glob('*GE*.shp'):
    country_code = file.name[:2]
    sf = shapefile.Reader(str(file))
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    GE_data[country_code] = pd.DataFrame(columns=fields, data=records)    
if not GE_data:
    print("No GE_data created. Check if .shp files are present in the expected pattern.")
else:
    print(f"Created GE_dataframes for {len(GE_data)} countries.")
    


Created GE_dataframes for 31 countries.


In [5]:
# merge the child, and geospatial recodes
BRGE_data = {}
for country in BR_data.keys():
    if country in GE_data:
        BRGE_data[country] = BR_data[country].merge(GE_data[country].set_index(['DHSCLUST', 'DHSCC']), right_index=True, left_on=['v001', 'country'], how='inner')
        print(f"Merging: {country}")
    else:
        print(f"No geolocation data to merge for country code: {country}")

print(f"Merged birth and geospatial recode data for {len(BRGE_data)} countries.")



Merging: MR
Merging: KE
Merging: ET
Merging: NG
Merging: ZW
Merging: UG
Merging: BU
Merging: BJ
Merging: GA
Merging: AO
Merging: MW
Merging: MZ
Merging: TZ
Merging: MD
Merging: ML
Merging: CI
Merging: CM
Merging: SL
Merging: CD
Merging: ZA
Merging: TD
Merging: GM
Merging: BF
Merging: GN
Merging: GH
Merging: LS
Merging: TG
Merging: SN
Merging: RW
Merging: LB
Merging: ZM
Merged birth and geospatial recode data for 31 countries.


In [6]:
# get the total number of births cases - all children born to the women being interviewed
births = []
for country in BRGE_data.keys():
    number_of_births = len(BRGE_data[country])
    print(f"{country}: {number_of_births}")
    births.append(number_of_births)

print(f"Total birth cases: {sum(births)}")

# get the number of columns for all countries' datasets
for country in BRGE_data.keys():
    number_of_columns = len(BRGE_data[country].columns)
    print(f"{country}: {number_of_columns}")

MR: 39793
KE: 77381
ET: 41392
NG: 127545
ZW: 20791
UG: 57906
BU: 45419
BJ: 45853
GA: 23232
AO: 42002
MW: 68074
MZ: 33900
TZ: 40394
MD: 47720
ML: 33379
CI: 39979
CM: 33988
SL: 40543
CD: 59276
ZA: 14144
TD: 68989
GM: 31665
BF: 48745
GN: 28887
GH: 34663
LS: 11710
TG: 26264
SN: 21562
RW: 30820
LB: 24765
ZM: 38446
Total birth cases: 1299227
MR: 1178
KE: 1338
ET: 1306
NG: 1221
ZW: 1133
UG: 1156
BU: 1155
BJ: 1207
GA: 1250
AO: 1179
MW: 1116
MZ: 1396
TZ: 1369
MD: 1181
ML: 1257
CI: 1183
CM: 1231
SL: 1192
CD: 1018
ZA: 1059
TD: 1071
GM: 1272
BF: 1157
GN: 1188
GH: 1267
LS: 1002
TG: 1073
SN: 1210
RW: 1235
LB: 1240
ZM: 1154


In [7]:
# Check which columns are different and count them
reference_columns = BRGE_data[list(BRGE_data.keys())[0]].columns

for country, df in BRGE_data.items():
    if set(df.columns) == set(reference_columns):
        print(f"{country}: Columns match")
    else:
        similar_columns = set(df.columns).intersection(set(reference_columns))
        missing_columns = set(reference_columns) - set(df.columns)
        additional_columns = set(df.columns) - set(reference_columns)
        print(f"{country}: Columns do not match")

        if similar_columns:
            print(f"    Similar columns ({len(similar_columns)}): {similar_columns}")
        if missing_columns:
            print(f"    Missing columns ({len(missing_columns)}): {missing_columns}")
        if additional_columns:
            print(f"    Additional columns ({len(additional_columns)}): {additional_columns}")

MR: Columns match
KE: Columns do not match
    Similar columns (942): {'v140', 'h21', 'hw56', 'm57a', 'h53d', 'v105', 'v525', 'ALT_GPS', 'v222', 'v469x', 'v028', 'v155', 'v413a', 'v411', 'v376', 'v745c', 'h8', 'v401', 'h12v', 'm57u', 'v104', 'v3a08f', 'v458', 'm2a', 'v007', 'v507', 'v414i', 'v223', 'v472a', 'h58d', 'h50', 'h62d', 'v481g', 'm2d', 'hw53', 'v394', 'idx94', 'v3a01', 'v045b', 'm57h', 'v414u', 'v743a', 'v3a00i', 'v404', 'v013', 'm45', 'h12w', 'h43', 'h12f', 'v204', 'v413', 'v014', 'h15j', 'v231', 'h53', 'hw70', 'm57v', 'm1', 'm57m', 'v3a08d', 'm57p', 'm19', 'b9', 'h12t', 'bidx', 'v001', 'v602', 'v633c', 'h37aa', 'h37z', 'v017', 'g122', 'v420', 'h6', 'h37c', 'awfacte', 'v153', 'm42d', 'v008', 'm8', 'v3a08ad', 'h13b', 'h62m', 'v731', 'v472h', 'h5', 'h15g', 'h14', 'h10', 'h63', 'h13', 'h11b', 'h57', 'v633g', 'v467c', 'v471e', 'm47', 'h66', 'v502', 'v310', 'h56', 'b18', 'v471b', 'v501', 'v504', 'v168', 'v191a', 'm39a', 'v319', 'hw18', 'v131', 'v631', 'v510', 'v3a08r', 'v3a00y', 

In [8]:
# concatenate countries' datasets - dropping all unsimilar columns
BRGE_concat = pd.concat(BRGE_data.values(), join='inner', ignore_index=True)
print(list(BRGE_concat.columns))
print(f"Number of columns: {len(BRGE_concat.columns)}")

['caseid', 'bidx', 'v000', 'v001', 'v002', 'v003', 'v004', 'v005', 'v006', 'v007', 'v008', 'v009', 'v010', 'v011', 'v012', 'v013', 'v014', 'v015', 'v016', 'v017', 'v018', 'v019', 'v019a', 'v020', 'v021', 'v022', 'v023', 'v024', 'v025', 'v026', 'v027', 'v028', 'v029', 'v030', 'v031', 'v032', 'v034', 'v040', 'v042', 'v044', 'v101', 'v102', 'v103', 'v104', 'v105', 'v106', 'v107', 'v113', 'v115', 'v116', 'v119', 'v120', 'v121', 'v122', 'v123', 'v124', 'v125', 'v127', 'v128', 'v129', 'v130', 'v131', 'v133', 'v134', 'v135', 'v136', 'v137', 'v138', 'v139', 'v140', 'v141', 'v149', 'v150', 'v151', 'v152', 'v153', 'awfactt', 'awfactu', 'awfactr', 'awfacte', 'awfactw', 'v155', 'v156', 'v157', 'v158', 'v159', 'v160', 'v161', 'v166', 'v167', 'v168', 'v190', 'v191', 'ml101', 'v201', 'v202', 'v203', 'v204', 'v205', 'v206', 'v207', 'v208', 'v209', 'v210', 'v211', 'v212', 'v213', 'v214', 'v215', 'v216', 'v217', 'v218', 'v219', 'v220', 'v221', 'v222', 'v223', 'v224', 'v225', 'v226', 'v227', 'v228', 'v22

In [9]:
# Drop columns that are completely NaN
columns_before = BRGE_concat.shape[1]
BRGE_concat = BRGE_concat.dropna(axis='columns', how='all')
columns_after = BRGE_concat.shape[1]
columns_dropped = columns_before - columns_after
print(f"Number of columns dropped: {columns_dropped}")  
print(f"Number of columns left: {columns_after}")


Number of columns dropped: 71
Number of columns left: 723


In [10]:
# removing observations that have missing for the midx - children not born within the past 5 years
def drop_rows_with_na_in_midx(df):
    rows_before = len(df)
    modified_df = df.dropna(subset=['midx'])
    rows_after = len(modified_df)
    
    rows_dropped = rows_before - rows_after
    if rows_dropped > 0:
        print(f"Dropped {rows_dropped} rows where 'midx' was NaN.")
        print(f"Left with {rows_after} rows representing children born in the past 5 years.")
    else:
        print("No rows were dropped as 'midx' had no NaN values.")
    
    return modified_df

BRGE_concat_modified = drop_rows_with_na_in_midx(BRGE_concat)

Dropped 974386 rows where 'midx' was NaN.
Left with 324841 rows representing children born in the past 5 years.


In [11]:
# pickling the BRGE_modified file
f = open('Processed_Datasets/BR_processing.pkl', 'wb')
pickle.dump(BRGE_concat_modified, f)

In [12]:
# pickling the metadata file
f = open('Processed_Datasets/BR_processing_metadata.pkl', 'wb')
pickle.dump(BR_metadata, f)
f.close()